## ATHENA: Residue-Level IDR Classification Tutorial
This Google Colab notebook provides a step-by-step guide for running the ATHENA residue-level (per-amino-acid) classifier (ATHENA_IDR_classification.py).

This pipeline operates in two main stages:

1. **Embedding Generation:** It uses an ESM model (e.g., esmc_300m) to generate per-residue embeddings for each protein in your FASTA file.

2. **IDR Prediction:** It feeds these embeddings into the trained Bi-LSTM + Transformer model to predict a disorder score and label for every single residue.

### Expected Runtime

**Anywhere from 1-10 minutes**, depending on the number of sequences to be classified.

### Step 1: Clone Repository and Set Up Environment
First, we will clone the ATHENA GitHub repository, move into the new directory, and install the required Python libraries. This includes the esm library, which is required for generating protein embeddings.

In [16]:
# 1. Clone the Repository

!git clone https://github.com/Shimizu-team/ATHENA
%cd ATHENA

# 2. Install Dependencies
!pip install torch pandas esm httpx tabulate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'ATHENA'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Receiving objects: 100% (4/4), done.
/Users/koheiota/Desktop/251028_ATHENA/ATHENA_IDR_Classifier/ATHENA


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Step 2: Prepare Python Scripts
The main script ATHENA_IDR_classification.py imports the model architecture from a file named Transformer_LSTM.py. The repository contains this model in ATHENA_IDR_Model.py.

We rename this file before we import the modules.

In [2]:
# Rename Model Script for Import
import os

model_source = "ATHENA_IDR_Model.py"
model_target = "Transformer_LSTM.py"

if os.path.exists(model_source):
    !mv {model_source} {model_target}
    print(f"Renamed '{model_source}' to '{model_target}' for import.")
else:
    print(f"Could not find '{model_source}'. Did the git clone fail or was the file misnamed?")
    

✅ Renamed 'ATHENA_IDR_Model.py' to 'Transformer_LSTM.py' for import.


### Step 3: Import Libraries
We import the necessary modules.

In [9]:
# Import Libraries
import os
import sys
import torch
import pandas as pd
import argparse

# Import the main function
try:
    # We rename the import for clarity
    from ATHENA_IDR_classification import main as run_idr_prediction
except ImportError as e:
    print(f"⚠️ Error importing modules: {e}")
    print("Please ensure 'Transformer_LSTM.py' (renamed from 'ATHENA_IDR_Model.py') is present.")

print("✅ Core modules imported successfully.")

✅ Core modules imported successfully.


### Step 4: Prepare Model Parameters
Now we will unzip the model weights.


In [10]:
# Define the directory and the zip file
model_dir = "ATHENA_IDR_model_params"
zip_filename = "ATHENA_IDR_Weights.pt.zip"
zip_filepath = os.path.join(model_dir, zip_filename)

print(f"Attempting to unzip '{zip_filepath}' into its parent directory '{model_dir}'...")

# Check if the zip file exists at the specified path
if not os.path.isfile(zip_filepath):
    print(f"\n⚠️ Error: File not found at '{zip_filepath}'.")
    print("Please ensure the GitHub repository was cloned correctly and this file exists.")
else:
    print(f"\nExtracting '{zip_filepath}'...")
    
    !unzip -q -o "{zip_filepath}" -d "{model_dir}"
    
    print(f"Unzip command finished. Files extracted to '{model_dir}'.")
    

Attempting to unzip 'ATHENA_IDR_model_params/ATHENA_IDR_Weights.pt.zip' into its parent directory 'ATHENA_IDR_model_params'...

Extracting 'ATHENA_IDR_model_params/ATHENA_IDR_Weights.pt.zip'...
Unzip command finished. Files extracted to 'ATHENA_IDR_model_params'.


### Step 5: Define Run Configuration
The original `ATHENA_IDR_classification.py` script is designed to be run from the command line. We define the run configurations here. 

We designate the input fasta file using the --fasta_file config.

In [11]:
# Colab Cell 6: Define Configuration (via sys.argv)
import sys

# These arguments match the 'README' and the script defaults
args_list = [
    'ATHENA_IDR_classification.py',
    
    '--fasta_file',
    'input/example_sequences.fasta',
    
    '--idr_model_path',
    'ATHENA_IDR_model_params/ATHENA_IDR_Weights.pt', 
    
    '--output_csv',
    'output/IDR_predictions.csv',
    
    '--esm_model_name',
    'esmc_300m', 
    
    '--batch_size',
    '8'
]

# Set sys.argv
sys.argv = args_list

print("--- Running Configuration ---")
print(" ".join(args_list))
print("-----------------------------")

--- Running Configuration ---
ATHENA_IDR_classification.py --fasta_file input/example_sequences.fasta --idr_model_path ATHENA_IDR_model_params/ATHENA_IDR_Weights.pt --output_csv output/IDR_predictions.csv --esm_model_name esmc_300m --batch_size 8
-----------------------------


### Step 6: Run Prediction 
With all configuration set, we can now call the main function from the script. This will execute the full pipeline.

In [12]:
# Colab Cell 7: Execute Prediction
print("Starting End-to-End IDR Prediction")
print("STEP 1: Generating Protein Embeddings (this may take time)...")

try:
    # Call the main function we imported
    run_idr_prediction()
    print("\nPipeline Complete")

except ImportError:
    print("Error: 'EnhancedLSTMTransformerIDRPredictor' not found.")
    print("Please ensure Step 2 (renaming the .py file) completed successfully.")

--- Starting End-to-End IDR Prediction ---
STEP 1: Generating Protein Embeddings (this may take time)...
Using device: cpu
Start time: 2025-10-28T12:09:13.501583

--- STEP 1: Generating Protein Embeddings ---
▶ Loading FASTA file...
Loaded 3 sequences.
Loading local ESM model: esmc_300m


Fetching 4 files: 100%|██████████| 4/4 [00:28<00:00,  7.04s/it]


▶ Warming up model...
▶ Warm-up complete
Processing batch 1/1: Proteins 0 to 2


Processing  100%|████████████████████████| 3/3 [Elapsed: 00:02 | Remaining: 00:00] , Success=3 Fail=0 Retry=0


Batch stats: 3 successful, 0 failed

Embedding generation complete.
Successfully processed 3/3 proteins.

--- STEP 2: Loading IDR Prediction Model ---
Successfully loaded model from ATHENA_IDR_model_params/ATHENA_IDR_Weights.pt

--- STEP 3: Running IDR Prediction ---
--- Inference Complete ---
Created directory: output

--- STEP 4: Results Saved ---
Inference results for 3 proteins saved to 'output/IDR_predictions.csv'
End time: 2025-10-28T12:09:50.569750

--- Pipeline Complete ---


### Step 7 Review and Interpret Results
The script saves all predictions to a single CSV file. We load it with pandas to see the results.

The output contains one row for every amino acid in your input file.

In [15]:
# Colab Cell 8: Load and Display Results
import pandas as pd
import os

output_file = 'output/IDR_predictions.csv'

if os.path.exists(output_file):
    print(f"Loading results from '{output_file}'...")
    
    df = pd.read_csv(output_file)
    
    print(f"\nTotal residues predicted: {len(df)}")
    
    # Display the first 20 predictions
    print("\n--- First 20 Residue Predictions ---")
    print(df.head(20).to_markdown(index=False))
    
    # Display predictions for the start of the second protein
    print("\n--- Predictions for 'protein_sample_02' ---")
    print(df[df['Protein ID'] == 'protein_sample_02'].head(20).to_markdown(index=False))

else:
    print(f"⚠️ Output file '{output_file}' not found.")
    print("Please ensure Step 7 completed without errors.")

Loading results from 'output/IDR_predictions.csv'...

Total residues predicted: 419

--- First 20 Residue Predictions ---
| Protein ID   |   Residue Index |   Predicted Label |   Disordered Probability |
|:-------------|----------------:|------------------:|-------------------------:|
| P12345       |               0 |                 0 |                   0.2416 |
| P12345       |               1 |                 0 |                   0.2253 |
| P12345       |               2 |                 0 |                   0.1976 |
| P12345       |               3 |                 0 |                   0.1923 |
| P12345       |               4 |                 0 |                   0.185  |
| P12345       |               5 |                 0 |                   0.1789 |
| P12345       |               6 |                 0 |                   0.1776 |
| P12345       |               7 |                 0 |                   0.175  |
| P12345       |               8 |                 0 |    